In [ ]:
import os
import numpy as np
import pandas as pd
from scroutines.config_plots import *

In [ ]:
ddir = '/data/qlyu/v1/data/easifish/lt185/r1/proc'
n5dir = '/data/qlyu/v1/data/easifish/lt185/r1/stitching/export.n5'
f1 = os.path.join(ddir, 'roi.csv')
f2 = os.path.join(ddir, 'spotcount_intn.csv')

In [ ]:
df1 = pd.read_csv(f1, index_col=0)
df2 = pd.read_csv(f2, index_col=0)
df  = df1.join(df2)

In [ ]:
df

In [ ]:
cond = df['area'] < 1500
df = df[cond]

In [ ]:
# cell level stats
fig, axs = plt.subplots(1,4,figsize=(4*4,1*4))
ax = axs[0]
sns.histplot(df['x'].values, ax=ax)
ax.set_xlabel('x')
ax = axs[1]
sns.histplot(df['y'].values, ax=ax)
ax.set_xlabel('y')
ax = axs[2]
sns.histplot(df['z'].values, ax=ax)
ax.set_xlabel('z')
ax = axs[3]
sns.histplot(df['area'].values, ax=ax)
ax.set_xlabel('area')
fig.tight_layout()
plt.show()

In [ ]:
channels = [
    'r1_c0',  'r1_c2',
]
genes = [
    'RL (Cre)',
    'LM (FlpO)',
]

In [ ]:
# cell level stats
for ch in channels: 
    fig, axs = plt.subplots(1,3,figsize=(3*4,1*4))
    ax = axs[0]
    sns.scatterplot(data=df, x='x', y=ch, ax=ax, s=3, edgecolor='none')
    ax.set_xlabel('x')

    ax = axs[1]
    sns.scatterplot(data=df, x='y', y=ch, ax=ax, s=3, edgecolor='none')
    ax.set_xlabel('y')

    ax = axs[2]
    sns.scatterplot(data=df, x='z', y=ch, ax=ax, s=3, edgecolor='none')
    ax.set_xlabel('z')

    fig.tight_layout()
    plt.show()

In [ ]:
df.describe()

In [ ]:
fig, ax = plt.subplots()
sns.histplot(df['r1_c0'].values, cumulative=True, element='step', fill=False, label='LM', ax=ax)
sns.histplot(df['r1_c2'].values, cumulative=True, element='step', fill=False, label='RL', ax=ax)
ax.axvline(5, linestyle='--', color='k')
ax.axvline(10, linestyle='--', color='k')
ax.axvline(30, linestyle='--', color='k')
ax.legend()

In [ ]:
for ch, gene in zip(channels, genes):
    fig, ax = plt.subplots(figsize=(10,10))
    vmax = np.percentile(df[f'{ch}'], 99.9)
    print(vmax)
    g = ax.scatter(df['x'].values, df['z'].values, c=df[f'{ch}'], s=1, edgecolor='none', 
                   vmax = vmax,
               cmap='rocket_r')
    ax.set_title(gene)
    ax.set_aspect('equal')
    fig.colorbar(g, shrink=0.3)
    plt.show()

In [ ]:
def rename(code):
    rename_dict = {
        0: 'NA', 
        1: 'RL', 
        2: 'LM', 
        3: 'both'}
    return rename_dict[code]

df = df.copy()
th = 10 
df['bi_r1_c0'] = df['r1_c0'] > th
df['bi_r1_c2'] = df['r1_c2'] > th 
df['lbl'] = (df['r1_c0'] > th).astype(int) + 2*(df['r1_c2'] > th).astype(int)
df['lbl_v2'] = df['lbl'].apply(rename)
np.unique(df['lbl'].values)
np.unique(df['lbl_v2'].values)

In [ ]:
for ch, gene in zip(channels, genes):
    
    fig, ax = plt.subplots(figsize=(10,10))
    g = ax.scatter(df['x'].values, df['z'].values, c=df[f'bi_{ch}'], s=3, edgecolor='none', 
               cmap='rocket_r')
    ax.set_title(gene)
    ax.set_aspect('equal')
    fig.colorbar(g, shrink=0.3)
    plt.show()

In [ ]:
sns.color_palette('tab10', 10)

In [ ]:
palette = {
    'NA': 'lightgray', 
    'LM': 'C3', 
    'RL': 'C0', 
    'both': 'C6', 
}

In [ ]:
df.value_counts('lbl_v2')

In [ ]:
a = df.value_counts('lbl_v2')
print(a/len(df)*100, "\n")

print(f"both in LM (%): {a.loc['both']/(a.loc['both']+a.loc['LM'])*100:.1f}")
print(f"both in RL (%): {a.loc['both']/(a.loc['both']+a.loc['RL'])*100:.1f}")
print(f"RL/LM (%): {(a.loc['both']+a.loc['RL'])/(a.loc['both']+a.loc['LM'])*100:.1f}")

In [ ]:
fig, ax = plt.subplots(figsize=(3,4))
df[df['lbl_v2']!='NA'].value_counts('lbl_v2').plot.bar(width=0.7, color='black')
plt.show()

In [ ]:
def plot_easifish_proj(data, x, y, hue='lbl_v2', na_rep='NA', invert_yaxis=False, figsize=(8,8)):
    
    fig, ax = plt.subplots(figsize=figsize)
    sns.scatterplot(data=df, 
                    x=x, y=y, color='lightgray', s=2, edgecolor='none')
    sns.scatterplot(data=df[df[hue]!=na_rep], 
                    x=x, y=y, hue=hue, palette=palette, s=5, edgecolor='none')
    if invert_yaxis:
        ax.invert_yaxis()
        
    ax.grid(False)
    ax.set_aspect('equal')
    ax.legend(bbox_to_anchor=(1,1))
    plt.show()

In [ ]:
plot_easifish_proj(df, 'x', 'y')
plot_easifish_proj(df, 'x', 'z', invert_yaxis=True)
plot_easifish_proj(df, 'y', 'z', invert_yaxis=True)

# check dimensions - make sure I understand them

#### Load 'n5' image data - cytoDAPI

In [ ]:
import zarr

In [ ]:
subpath='c3/s3' # channel and scale, full resolution (s0) image can be loaded for detailed inspection
im = zarr.open(store=zarr.N5Store(n5dir), mode='r')
img = im[subpath] ## can also choose to load a subset of the image data
img.shape # z, y, x

In [ ]:
img = img[:,:,:]

In [ ]:
np.percentile(img, [0,5,50,95, 99, 99.5, 100])

In [ ]:
img2d = img[:,400,:]
vmin, vmax = np.percentile(img2d, [5,99])
print(vmin, vmax)

fig, ax = plt.subplots(figsize=(10,8))
g = ax.imshow(img2d, vmin=vmin, vmax=vmax, cmap='rocket')
ax.grid(False)
# ax.axis('off')
ax.set_aspect('equal')
# fig.colorbar(g, shrink=0.3)
plt.show()

In [ ]:
for cut in np.arange(50,900,200):
    img2d = img[:,cut,:]
    vmin, vmax = np.percentile(img2d, [5,99])
    print(vmin, vmax)

    fig, ax = plt.subplots(figsize=(10,8))
    g = ax.imshow(img2d, vmin=vmin, vmax=vmax, cmap='rocket')
    ax.grid(False)
    # ax.axis('off')
    ax.set_aspect('equal')
    # fig.colorbar(g, shrink=0.3)
    plt.show()

In [ ]:
img2d = img[:,400,:]
vmin, vmax = np.percentile(img2d, [5,99])
print(vmin, vmax)

fig, ax = plt.subplots(figsize=(10,8))
g = ax.imshow(img2d, vmin=vmin, vmax=vmax, cmap='rocket')
ax.grid(False)
# ax.axis('off')
ax.set_aspect('equal')
# fig.colorbar(g, shrink=0.3)
plt.show()

In [ ]:
img2d = img[400,:,:]
vmin, vmax = np.percentile(img2d, [5,99])
print(vmin, vmax)

fig, ax = plt.subplots(figsize=(10,8))
g = ax.imshow(img2d, vmin=vmin, vmax=vmax, cmap='rocket')
ax.grid(False)
# ax.axis('off')
ax.set_aspect('equal')
# fig.colorbar(g, shrink=0.3)
plt.show()

In [ ]:
subpath0='c0/s3' # channel and scale, full resolution (s0) image can be loaded for detailed inspection
img0 = im[subpath0] ## can also choose to load a subset of the image data
img0 = img0[:,:,:]
np.percentile(img0, [0,5,50,95, 99, 99.5, 100])

In [ ]:
img2d = np.max(img0[:,:,:], axis=2)
vmin, vmax = np.percentile(img2d, [5,100])
print(vmin, vmax)

fig, ax = plt.subplots(figsize=(10,8))
g = ax.imshow(img2d, vmin=vmin, vmax=vmax, cmap='rocket')
ax.grid(False)
# ax.axis('off')
ax.set_aspect('equal')
# fig.colorbar(g, shrink=0.3)
plt.show()

In [ ]:
subpath0='c2/s3' # channel and scale, full resolution (s0) image can be loaded for detailed inspection
img0 = im[subpath0] ## can also choose to load a subset of the image data
img0 = img0[:,:,:]
np.percentile(img0, [0,5,50,95, 99, 99.5, 100])

In [ ]:
img2d = np.max(img0[:,:,:], axis=2)
vmin, vmax = np.percentile(img2d, [5,100])
print(vmin, vmax)

fig, ax = plt.subplots(figsize=(10,8))
g = ax.imshow(img2d, vmin=vmin, vmax=vmax, cmap='rocket')
ax.grid(False)
# ax.axis('off')
ax.set_aspect('equal')
# fig.colorbar(g, shrink=0.3)
plt.show()